In [9]:
import requests
import json
import pandas as pd
import logging
import time
import os
from tqdm import tqdm

### General Set Up


In [10]:
logging.basicConfig(filename='nexus_mods_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

API_KEY = "P4WFtQQ8zMc89xCESY0x2E183bfOpHRjvY159e4H3HcOJ6GNaDI=--I3/cG6WtrxR9ac4t--/e+gBW1nRPXJS7PggzSspg=="
HEADERS = {
    'apikey': API_KEY,
    'Accept': 'application/json'
}
BASE_URL = "https://api.nexusmods.com/v1/"

REQUEST_LIMIT = 2500
REQUESTS_PER_HOUR_LIMIT = 100

OUTPUT_FILE = "nexus_mods_games.csv"
CHECKPOINT_FILE = "checkpoint.json"

#### Function for API limits

In [11]:
# Function to handle API rate limits
def rate_limited_request(url, params=None):
    while True:
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code == 429:
            logging.warning("Rate limit reached. Sleeping for 60 seconds.")
            time.sleep(60)
        else:
            return response

#### Functions for checkpoints

In [12]:
# Load checkpoint
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return json.load(f)
    return {}

In [13]:
# Save checkpoint
def save_checkpoint(data):
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(data, f)

#### Games List and Details